本书代码：https://github.com/backstopmedia/tensorflowbook

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### 1. Tensor, Operation, Session对象

In [20]:
a = tf.constant(5, name='input_a')
b = tf.constant(3, name='input_b')
c = tf.multiply(a,b, name='mul_c')
d = tf.add(a,b, name='add_d')
e = tf.add(c,d, name='add_e')
sess = tf.Session()
ans = sess.run(e)
print(ans)
sess.run(c)

23


15

In [4]:
writer = tf.summary.FileWriter('./my_graph', sess.graph)

In [ ]:
!tensorboard --logdir='./my_graph'

In [23]:
sess.close()
writer.close()

In [59]:
a = tf.constant([3,5,7], name='input_a')
c = tf.reduce_sum(a, name='mul_c')
d = tf.reduce_prod(a, name='add_d')
e = tf.add(c,d, name='add_e')
sess = tf.Session()
ans = sess.run(e)
print(ans)

120


In [60]:
print(sess.run(tf.shape(a)))
sess.run([a,c,d,e])

[3]


[array([3, 5, 7]), 15, 105, 120]

In [66]:
replace_dict = {c: 150}
sess.run([a,c,d,e], feed_dict=replace_dict)
# feed_dict是覆盖，即使c的计算结果是15，也会被覆盖

[array([3, 5, 7]), array(150), 105, 255]

In [67]:
sess.close()

TensorFlow Operation也称Op， 是一些对（或利用） Tensor对象执行运算的节点。 计算完毕后， 它们会返回0个或多个张量， 可在以后为数据流图中的其他Op所使用。 为创建Op， 需要在Python中调用其构造方法。 调用时， 需要传入计算所需的所有Tensor参数（称为输入） 以及为正确创建Op的任何附加信息（称为属性） 。 Python构造方法将返回一个指向所创建Op的输出（0个或多个Tensor对象） 的句柄。 能够传递给其他Op或Session.run

句柄是什么？指针吗?

### 2. placehodler对象

In [56]:
print([np.float16 == tf.float16, np.float32 == tf.float32, np.float64 == tf.float64, np.bool==tf.bool])
print([np.int8 == tf.int8, np.int16 == tf.int16, np.int32 == tf.int32, np.int64 == tf.int64])
print([tf.uint8, tf.string, tf.qint8])

[True, True, True, True]
[True, True, True, True]
[tf.uint8, tf.string, tf.qint8]


In [82]:
a = tf.placeholder(dtype=tf.float32, shape=[3], name='input')
print(a.dtype, a.shape, a.name)
b = tf.reduce_sum(a)
c = tf.reduce_prod(a)
d = tf.add(b,c)

<dtype: 'float32'> (3,) input_6:0


dtype参数必须， shape参数可选,默认为None，任意维度任意长度， name参数可选。 笔者强烈建议你养成显式声明Tensor对象的数值属性的习惯，因为当处理的流图规模较大时，相信你一定不希望去逐一排查到底哪些对象导致了TypeMismatchError！

In [86]:
input_dict = {a: np.array([3,4,6], dtype=np.float32)}
with tf.Session() as sess:
    print(sess.run([a,b,c,d], feed_dict=input_dict))

[array([3., 4., 6.], dtype=float32), 13.0, 72.0, 85.0]


tf.placeholder是Operation。tensorflow中，Tensor和Operation是immutable，Variable对象是可变的。

In [102]:
sess = tf.Session()
print( sess.run(tf.zeros((2,3))) )
print( sess.run(tf.ones((2,3))) )
print( sess.run(tf.random_normal((2,3))) )
print( sess.run(tf.random_uniform((2,3))) )
print( sess.run(tf.truncated_normal((2,3), mean=5.0, stddev=1.0)) )#不会有小于3或大于7的元素
#不会创建任何偏离均值超过2倍标准差的值，从而可以防止有一个或两个元素与该张量中的其他元素显著不同的情况出现
sess.close()

[[0. 0. 0.]
 [0. 0. 0.]]
[[1. 1. 1.]
 [1. 1. 1.]]
[[-1.0554614  -0.99829733  1.4435314 ]
 [-0.0977305  -1.0170534   2.9315066 ]]
[[0.7331735  0.29038632 0.18845737]
 [0.6761904  0.05591452 0.29866135]]
[[3.8828673 5.9375644 4.557488 ]
 [5.055162  4.4379168 5.8577147]]


### 3. Variable对象

为使用Variable对象，需要采取一些额外的步骤——必须在一个Session对象内对Variable对象进行初始化。 这样会使Session对象开始追踪这个Variable对象的值的变化。 Variable对象的初始化通常是通过将tf.global_variables_initializer()这个Op传给Session.run()完成的

In [137]:
my_var = tf.Variable([1,2,3], dtype=tf.float32, name='my_var')

In [138]:
init = tf.global_variables_initializer()# this is op
# 如果只对一个Variable初始化
# tf.initialize_variables([my_var])

In [143]:
my_var_times_two = my_var.assign(my_var*2)# this is op!
# my_var.assign_add，自加
# my_var.assign_sub，自减

In [140]:
sess = tf.Session()
sess.run(init)

In [145]:
print(sess.run(my_var_times_two))
print(sess.run(my_var))

[16. 32. 48.]
[16. 32. 48.]


In [146]:
sess.close()

不同Session对象会各自独立地维护Variable对象的值， 因此每个Session对象都拥有自己的、 在Graph对象中定义的Variable对象的当前值

In [148]:
sess1 = tf.Session()
sess2 = tf.Session()

In [168]:
sess1.run(init)
for i in range(4):
    print(sess1.run(my_var_times_two))
sess1.run(my_var_times_two)

[2. 4. 6.]
[ 4.  8. 12.]
[ 8. 16. 24.]
[16. 32. 48.]


array([32., 64., 96.], dtype=float32)

In [169]:
sess2.run(init)
for i in range(2):
    print(sess2.run(my_var_times_two))
sess2.run(my_var_times_two)

[2. 4. 6.]
[ 4.  8. 12.]


array([ 8., 16., 24.], dtype=float32)

In [170]:
sess1.close()
sess2.close()

### 4. grap对象

下面来研究TensorFlow的Graph对象， 学习如何创建更多的数据流图， 以及如何让多个流图协同工作。

在大多数TensorFlow程序中， 只使用默认数据流图就足够了。 然而， 如果需要定义多个相互之间不存在依赖关系的模型， 则创建多个Graph对象十分有用。

此外， 从其他TensorFlow脚本中加载之前定义的模型， 并利用Graph.as_graph_def（） 和tf.import_graph_def（） 函数将其赋给Graph对象也是可行的。 这样， 用户便可在同一个Python文件中计算和使用若干独立的模型的输出。 本书后续章节将介绍数据流图的导入和导出。

### 5. trainable参数

在本书的后续章节将介绍各种能够自动训练机器学习模型的Optimizer类，这意味着这些类将自动修改Variable对象的值，而无须显式做出请求。在大多数情况下，这与读者的期望一致，但如果要求Graph对象中的一些Variable对象只可手工修改，而不允许使用Optimizer类时，可在创建这些Variable对象时将其trainable参数设为False。

In [177]:
no_trainable_variable = tf.Variable([2,3,4], trainable=False)
no_trainable_variable

<tf.Variable 'Variable_2:0' shape=(3,) dtype=int32_ref>

### 6. 名称作用域

现在开始介绍构建任何TensorFlow数据流图所必需的核心构件。到目前为止，我们只接触了包含少量节点和阶数较小的张量的非常简单的数据流图，但现实世界中的模型往往会包含几十或上百个节点，以及数以百万计的参数。为使这种级别的复杂性可控，TensorFlow当前提供了一种帮助用户组织数据流图的机制——名称作用域（name scope）。名称作用域非常易于使用， 且在用TensorBoard对Graph对象可视化时极有价值。
https://github.com/backstopmedia/tensorflowbook

In [200]:
graph = tf.Graph()

with graph.as_default():
    in_1 = tf.placeholder(tf.float32, shape=[], name="input_a")
    in_2 = tf.placeholder(tf.float32, shape=[], name="input_b")
    const = tf.constant(3, dtype=tf.float32, name="static_value")

    with tf.name_scope("Transformation"):

        with tf.name_scope("A"):
            A_mul = tf.mul(in_1, const)
            A_out = tf.sub(A_mul, in_1)

        with tf.name_scope("B"):
            B_mul = tf.mul(in_2, const)
            B_out = tf.sub(B_mul, in_2)

        with tf.name_scope("C"):
            C_div = tf.div(A_out, B_out)
            C_out = tf.add(C_div, const)

        with tf.name_scope("D"):
            D_div = tf.div(B_out, A_out)
            D_out = tf.add(D_div, const)

    out = tf.maximum(C_out, D_out)   

writer = tf.summary.FileWriter('./name_scope_1', graph=graph)
writer.close()

AttributeError: module 'tensorflow' has no attribute 'mul'

### 7. over exercise

下面通过一个综合运用了之前讨论过的所有组件——**Tensor**、**Graph**、**Operation**、**Variable**、**placeholder**、**Session**以及**名称作用域**的练习来结束本章。练习结束后，读者将能够自如地搭建基本的TensorFlow数据流图并在TensorBoard中对其进行研究。